In [ ]:
import pandas as pd
import sqlite3

In [ ]:
import os

print("Daftar file di /content/:")
print(os.listdir("/content/"))

Daftar file di /content/:
['.config', 'Online Retail Store - customer.xlsx', 'Online Retail Store - sales.xlsx', 'Online Retail Store - product.xlsx', 'sample_data']


In [ ]:
files = [
    'Online Retail Store - customer.xlsx',
    'Online Retail Store - sales.xlsx',
    'Online Retail Store - product.xlsx'
]

for file_path in files:
    print(f"Data Dari {file_path}:")
    df = pd.read_excel(file_path)
    print(df.head())
    print("\n" + "="*50 + "\n")


Data Dari Online Retail Store - customer.xlsx:
   CustomerID     Name  Surname  Loyalty_Discount
0   100000001    Simon    Walsh              0.10
1   100000003     Liam    Brown              0.09
2   100000005  Deirdre  Pullman              0.10
3   100000007  Dorothy  Thomson              0.07
4   100000010  Dominic     Parr              0.00


Data Dari Online Retail Store - sales.xlsx:
   TransactionID Order_Date Arrival_Date  CustomerID Shipping_State  \
0         113252 2003-07-03   2003-07-11   100000007        Alabama   
1         113248 2003-11-15   2003-11-21   400000706        Alabama   
2         113208 2004-12-03   2004-12-09   100000121        Alabama   
3         113138 2004-03-03   2004-03-09   100000091        Alabama   
4         113047 2004-12-21   2004-12-22   100000332        Alabama   

      Item_id  Quantity  Rating  
0  6697166886         2       4  
1  1893735141         5       3  
2  2363094138         1       5  
3  8948840620         9       5  
4  1284609

**NO 1**

In [ ]:
sales = pd.read_excel('Online Retail Store - sales.xlsx')
customer = pd.read_excel('Online Retail Store - customer.xlsx')
product = pd.read_excel('Online Retail Store - product.xlsx')

In [ ]:
merged_data = pd.merge(sales, product, on='Item_id')
merged_data['total_sales'] = merged_data['Quantity'] * merged_data['Retail_Price']
merged_data['Order_Date'] = pd.to_datetime(merged_data['Order_Date'], errors='coerce')
merged_data['year'] = merged_data['Order_Date'].dt.year
merged_data['month'] = merged_data['Order_Date'].dt.month

monthly_sales = merged_data.groupby(['year', 'month'])['total_sales'].sum().reset_index()

monthly_sales['prev_month_sales'] = monthly_sales['total_sales'].shift(1)
monthly_sales['sales_growth'] = (
    (monthly_sales['total_sales'] - monthly_sales['prev_month_sales']) /
    monthly_sales['prev_month_sales']
) * 100

monthly_sales['periode'] = monthly_sales['year'].astype(str) + '/' + monthly_sales['month'].astype(str).str.zfill(2)


result = monthly_sales[['periode', 'total_sales', 'prev_month_sales', 'sales_growth']]
print("Persentase Growth pada Total Sales per Bulan:")
print(result)


Persentase Growth pada Total Sales per Bulan:
    periode  total_sales  prev_month_sales  sales_growth
0   2003/01      9307.88               NaN           NaN
1   2003/02     13228.38           9307.88     42.120225
2   2003/03     21297.68          13228.38     60.999911
3   2003/04      9657.69          21297.68    -54.653793
4   2003/05     23963.28           9657.69    148.126415
5   2003/06     13674.98          23963.28    -42.933605
6   2003/07     24012.46          13674.98     75.594114
7   2003/08     12909.50          24012.46    -46.238328
8   2003/09     25439.48          12909.50     97.060150
9   2003/10     45836.16          25439.48     80.177268
10  2003/11    106138.17          45836.16    131.559908
11  2003/12     37527.78         106138.17    -64.642522
12  2004/01     21986.27          37527.78    -41.413348
13  2004/02     22263.75          21986.27      1.262060
14  2004/03     25730.24          22263.75     15.570108
15  2004/04     17552.02          25730.24

**NO 2**

In [ ]:
merged_data = pd.merge(sales, product, on='Item_id')
merged_data = pd.merge(merged_data, customer, on='CustomerID')
merged_data['spending'] = merged_data['Quantity'] * merged_data['Retail_Price']

filtered_data = merged_data[
    ~merged_data['Shipping_State'].isin(['Wyoming', 'Idaho', 'Alaska'])
]

customer_spending = filtered_data.groupby('CustomerID').agg(
    total_spending=('spending', 'sum'),
    total_quantity=('Quantity', 'sum')
).reset_index()

def categorize_spending(spending):
    if spending > 500:
        return 'High_spender'
    elif 300 <= spending <= 500:
        return 'Mid_spender'
    else:
        return 'Low_spender'

customer_spending['cust_cat'] = customer_spending['total_spending'].apply(categorize_spending)

summary = customer_spending.groupby('cust_cat').agg(
    total_user=('CustomerID', 'count'),
    avg_quantity=('total_quantity', 'mean'),
    avg_spending=('total_spending', 'mean')
).reset_index()

print("Kategori Pelanggan Berdasarkan Spending:")
print(summary)


Kategori Pelanggan Berdasarkan Spending:
       cust_cat  total_user  avg_quantity  avg_spending
0  High_spender         636     20.691824   1266.781698
1   Low_spender         148      5.425676    159.768649
2   Mid_spender         106      9.320755    404.678679


**NO 3**

In [ ]:
merged_data = pd.merge(sales, product, on='Item_id')
merged_data['Order_Date'] = pd.to_datetime(merged_data['Order_Date'], errors='coerce')

data_2004 = merged_data[merged_data['Order_Date'].dt.year == 2004]
product_ratings = data_2004.groupby('Description').agg(
    avg_rating=('Rating', 'mean')
).reset_index()

product_ratings['Ranking'] = product_ratings['avg_rating'].rank(ascending=False, method='first')

top_5 = product_ratings.nsmallest(5, 'Ranking')
bottom_5 = product_ratings.nlargest(5, 'Ranking')

result = pd.concat([top_5, bottom_5]).sort_values(by='Ranking')

result['Year'] = 2004

print("Ranking Produk pada Tahun 2004:")
print(result[['Year', 'Description', 'avg_rating', 'Ranking']])


Ranking Produk pada Tahun 2004:
     Year          Description  avg_rating  Ranking
31   2004         Crib Sheet A    4.600000      1.0
7    2004           Bath Mat C    4.500000      2.0
76   2004               Sheets    4.500000      3.0
122  2004          Washcloth A    4.444444      4.0
49   2004         Hand Towel A    4.428571      5.0
36   2004              Dress A    3.500000    121.0
100  2004  Suit & 2 pc Dress C    3.500000    122.0
120  2004            Valance B    3.500000    123.0
77   2004              Shirt A    3.444444    124.0
105  2004            Sweater C    3.444444    125.0
